In [2]:
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

True

# Aircraft category

In [3]:
import pandas as pd
import os

source_data_folder=os.getenv("SOURCE_FOLDER")
challenge_file=os.getenv("CHALLENGE_FILE")
challenge_file_preproc=os.getenv("CHALLENGE_FILE_PREPROC")
submission_file=os.getenv("SUBMISSION_FILE")
submission_file_preproc=os.getenv("SUBMISSION_FILE_PREPROC")

df = pd.read_csv(os.path.join(source_data_folder,challenge_file))
df_submission = pd.read_csv(os.path.join(source_data_folder,submission_file))

# Mapping of aircraft_type, and categories
weight_class_mapping = {
    'A332': 'Heavy', 'A333': 'Heavy', 'A343': 'Heavy',
    'A359': 'Heavy', 'B772': 'Heavy', 'B773': 'Heavy',
    'B77W': 'Heavy', 'B788': 'Heavy', 'B789': 'Heavy',
    'B752': 'Heavy', 'B763': 'Heavy', 'A310': 'Heavy',
    'A20N': 'Medium', 'A21N': 'Medium', 'A319': 'Medium', 'A320': 'Medium',
    'A321': 'Medium', 'B738': 'Medium', 'B739': 'Medium', 'B38M': 'Medium',
    'B39M': 'Medium', 'B737': 'Medium',
    'BCS1': 'Light', 'BCS3': 'Light', 'CRJ9': 'Light', 'E190': 'Light',
    'E195': 'Light', 'E290': 'Light', 'AT76': 'Light',
    'C56X': 'Light'
}

# Add "aircraft_category" column
df['aircraft_category'] = df['aircraft_type'].map(weight_class_mapping)
df_submission['aircraft_category'] = df_submission['aircraft_type'].map(weight_class_mapping)

# Check if all aircraft_type exist
df_missing = df[df['aircraft_category'].isna()]
if not df_missing.empty:
    print("The following aircraft_types does not exist in mapping :")
    print(df_missing['aircraft_type'].unique())

# Check if all aircraft_type exist
df_sub_missing = df_submission[df_submission['aircraft_category'].isna()]
if not df_sub_missing.empty:
    print("The following aircraft_types does not exist in mapping :")
    print(df_sub_missing['aircraft_type'].unique())

df.to_csv(os.path.join(source_data_folder,challenge_file_preproc), index=False)
df_submission.to_csv(os.path.join(source_data_folder,submission_file_preproc), index=False)


In [4]:
import pandas as pd

# Load File
df_aircraft = pd.read_csv(os.path.join(source_data_folder,os.getenv("AIRCRAFT_DB")))
df_test = pd.read_csv(os.path.join(source_data_folder,challenge_file_preproc))
df_submission = pd.read_csv(os.path.join(source_data_folder,submission_file_preproc))

# Mapping RECAT with aircraft_category
recat_mapping = {
    "CAT-A": "Heavy", 
    "CAT-B": "Heavy", 
    "CAT-C": "Heavy", 
    "CAT-D": "Medium", 
    "CAT-E": "Light", 
    "CAT-F": "Light"
}

# Update both files with RECAT
def update_aircraft_info(df_target, df_aircraft):
    for _, row in df_aircraft.iterrows():
        aircraft_type = row['Aircraft_Type']
        recat_value = row['RECAT-EU']
        
        # Chech if exists
        mask = df_target['aircraft_type'] == aircraft_type
        
        # If exists, update file
        if mask.any():
            df_target.loc[mask, 'aircraft_category'] = recat_mapping.get(recat_value, df_target.loc[mask, 'aircraft_category'])
            df_target.loc[mask, 'aircraft_OEW'] = row['Aircraft_OEW']
            df_target.loc[mask, 'aircraft_ZFW'] = row['Aircraft_ZFW']
            df_target.loc[mask, 'aircraft_max_range'] = row['Aircraft_MaxRange']
    
    return df_target

df_test = update_aircraft_info(df_test, df_aircraft)
df_submission = update_aircraft_info(df_submission, df_aircraft)

# Save changes to file
df_test.to_csv(os.path.join(source_data_folder,challenge_file_preproc), index=False)
df_submission.to_csv(os.path.join(source_data_folder,submission_file_preproc), index=False)


# MTOW, OEW

In [5]:
from openap import prop
import pandas as pd

# List of files to process
files = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Iterate through each file
for file in files:
    # Load the CSV file
    df_flight = pd.read_csv(file)

    # List of unique aircraft types in the file
    aircraft_types = df_flight['aircraft_type'].unique()

    # Initialize a dictionary to store MTOW and OEW values
    aircraft_data = {}

    # Use the openap library to retrieve information for each aircraft
    for aircraft_type in aircraft_types:
        try:
            # Get aircraft information via ICAO
            aircraft = prop.aircraft(f'{aircraft_type}')
            mtow = aircraft['mtow']
            oew = aircraft['oew']
            
            # Store results in the dictionary
            aircraft_data[aircraft_type] = {'max_tow': mtow, 'min_tow': oew}
        except Exception as e:
            print(f"Error for {aircraft_type}: {e}")
            aircraft_data[aircraft_type] = {'max_tow': None, 'min_tow': None}

    # Add 'min_tow' and 'max_tow' columns to the DataFrame
    df_flight['aircraft_OEW'] = df_flight['aircraft_type'].map(lambda x: aircraft_data.get(x, {}).get('min_tow', None))
    df_flight['aircraft_MTOW'] = df_flight['aircraft_type'].map(lambda x: aircraft_data.get(x, {}).get('max_tow', None))

    df_flight.to_csv(file, index=False)

    print(f"The columns min_tow and max_tow have been added and saved in '{file}'.")


Error for CRJ9: Aircraft crj9 not available.
Error for BCS3: Aircraft bcs3 not available.
Error for AT76: Aircraft at76 not available.
Error for BCS1: Aircraft bcs1 not available.
Error for A310: Aircraft a310 not available.
Error for C56X: Aircraft c56x not available.
Error for E290: Aircraft e290 not available.
The columns min_tow and max_tow have been added and saved in '/workspaces/PRCDataChallenge/data/challenge_set_preproc.csv'.
Error for BCS3: Aircraft bcs3 not available.
Error for CRJ9: Aircraft crj9 not available.
Error for BCS1: Aircraft bcs1 not available.
Error for AT76: Aircraft at76 not available.
Error for E290: Aircraft e290 not available.
The columns min_tow and max_tow have been added and saved in '/workspaces/PRCDataChallenge/data/submission_set_preproc.csv'.


In [6]:
import pandas as pd

# List of files to process
files = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Create a dictionary with the values of aircraft_OEW and aircraft_MTOW
tow_data = {
    'CRJ9': {'aircraft_OEW': 21772, 'aircraft_MTOW': 38329},
    'BCS3': {'aircraft_OEW': 37100, 'aircraft_MTOW': 70900},
    'AT76': {'aircraft_OEW': 13311, 'aircraft_MTOW': 21000},
    'A310': {'aircraft_OEW': 80000, 'aircraft_MTOW': 153000},
    'BCS1': {'aircraft_OEW': 37500, 'aircraft_MTOW': 63100},
    'C56X': {'aircraft_OEW': 5750, 'aircraft_MTOW': 9163},
    'E290': {'aircraft_OEW': 27853, 'aircraft_MTOW': 56400},
    'B763': {'aircraft_OEW': 95000, 'aircraft_MTOW': 187000},
    'B737': {'aircraft_OEW': 39000, 'aircraft_MTOW': 79000},
    'B788': {'aircraft_OEW': 118000, 'aircraft_MTOW': 228000},
    'B789': {'aircraft_OEW': 127000, 'aircraft_MTOW': 254000}
}

# Iterate through each file
for file in files:
    # Load the CSV file
    df_flight = pd.read_csv(file)

    # Fill the 'aircraft_OEW' and 'aircraft_MTOW' columns based on aircraft_type
    df_flight['aircraft_OEW'] = df_flight.apply(
        lambda row: tow_data.get(row['aircraft_type'], {}).get('aircraft_OEW', row.get('aircraft_OEW')),
        axis=1
    )
    df_flight['aircraft_MTOW'] = df_flight.apply(
        lambda row: tow_data.get(row['aircraft_type'], {}).get('aircraft_MTOW', row.get('aircraft_MTOW')),
        axis=1
    )

    # Save the modified file

    df_flight.to_csv(file, index=False)

    print(f"The columns 'aircraft_OEW' and 'aircraft_MTOW' have been updated for the corresponding aircraft types in '{file}'.")


The columns 'aircraft_OEW' and 'aircraft_MTOW' have been updated for the corresponding aircraft types in '/workspaces/PRCDataChallenge/data/challenge_set_preproc.csv'.
The columns 'aircraft_OEW' and 'aircraft_MTOW' have been updated for the corresponding aircraft types in '/workspaces/PRCDataChallenge/data/submission_set_preproc.csv'.


# Ratio TOW

In [7]:
import pandas as pd

# Load the CSV file
updated_tow_file=os.path.join(source_data_folder,challenge_file_preproc)
df_flight = pd.read_csv(updated_tow_file)

# Check if the columns 'tow', 'aircraft_OEW', and 'aircraft_MTOW' exist in the DataFrame
if all(col in df_flight.columns for col in ['tow', 'aircraft_OEW', 'aircraft_MTOW']):
    # Calculate the load_ratio according to the given formula
    df_flight['load_ratio'] = (df_flight['tow'] - df_flight['aircraft_OEW']) / (df_flight['aircraft_MTOW'] - df_flight['aircraft_OEW'])
    
    # Save the modified file
    df_flight.to_csv(updated_tow_file, index=False)
    
    print("The 'load_ratio' column has been added and saved to '{}'.".format(updated_tow_file))
else:
    print("The columns 'tow', 'aircraft_OEW', or 'aircraft_MTOW' are missing in the file.")


The 'load_ratio' column has been added and saved to '/workspaces/PRCDataChallenge/data/challenge_set_preproc.csv'.


In [8]:
import pandas as pd

# Load the CSV file
updated_tow_file=os.path.join(source_data_folder,submission_file_preproc)
df_flight = pd.read_csv(updated_tow_file)

df_flight['load_ratio'] = None

df_flight.to_csv(updated_tow_file, index=False)

print("The 'load_ratio' column has been added and saved to '{}'.".format(updated_tow_file))

The 'load_ratio' column has been added and saved to '/workspaces/PRCDataChallenge/data/submission_set_preproc.csv'.


# Flown_distance ratio

In [9]:
import pandas as pd
test_file_preproc=os.path.join(source_data_folder,challenge_file_preproc)
submission_file_preproc=os.path.join(source_data_folder,submission_file_preproc)
df_test = pd.read_csv(test_file_preproc)
df_submission = pd.read_csv(submission_file_preproc)

# Function to add the "ratio_flown_distance_max_range" column
def add_flown_distance_ratio(df_target):
    # Assume that 'flown_distance' and 'aircraft_max_range' exist
    df_target['ratio_flown_distance_max_range'] = df_target['flown_distance'] / df_target['aircraft_max_range']
    return df_target

# Add the "ratio_flown_distance_max_range" column
df_test = add_flown_distance_ratio(df_test)
df_submission = add_flown_distance_ratio(df_submission)

# Save the modifications to the CSV files
df_test.to_csv(test_file_preproc, index=False)
df_submission.to_csv(submission_file_preproc, index=False)

print("\nThe modified files have been saved.")



The modified files have been saved.


# Mode / Max / Median altitude and cruise level

In [10]:
import pandas as pd

# Read the CSV files
altitude_data = pd.read_csv(os.path.join(source_data_folder,os.getenv("TRAJECTORY_DATA")))
challenge_set_og = pd.read_csv(test_file_preproc)
final_submission_set = pd.read_csv(submission_file_preproc)

# Merge altitude_data with challenge_set_og on flight_id
merged_challenge_set = pd.merge(
    challenge_set_og,
    altitude_data[['flight_id','fl_mode','fl_max','fl_median', 'plateau_climb_rate_avg', 'plateau_altitude',
                   'ground_airspeed','ground_wind_speed','ground_wind_direction','ground_airspeed_angle',
                   'plateau_airspeed','plateau_wind_speed','plateau_wind_direction','plateau_airspeed_angle'
                   ]],
    on='flight_id',
    how='left'
)

# Merge altitude_data with final_submission_set on flight_id
merged_final_submission_set = pd.merge(
    final_submission_set,
    altitude_data[['flight_id','fl_mode','fl_max','fl_median', 'plateau_climb_rate_avg', 'plateau_altitude',
                   'ground_airspeed','ground_wind_speed','ground_wind_direction','ground_airspeed_angle',
                   'plateau_airspeed','plateau_wind_speed','plateau_wind_direction','plateau_airspeed_angle'
                   ]],
    on='flight_id',
    how='left'
)

# Save the results to new CSV files
merged_challenge_set.to_csv(test_file_preproc, index=False)
merged_final_submission_set.to_csv(submission_file_preproc, index=False)

print("\nThe files have been merged and saved.")



The files have been merged and saved.


# Combo Adep / Ades

In [11]:
import pandas as pd

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Loop through each file
for file in files_to_process:
    # Load the CSV file into a DataFrame
    df_flight = pd.read_csv(file)

    # Create a new column 'adep_ades' that combines 'adep' and 'ades'
    df_flight['adep_ades'] = df_flight[['adep', 'ades']].apply(lambda x: f'{x[0]}_{x[1]}', axis=1)

    # Assign a unique numeric value to each 'adep_ades' combination
    df_flight['adep_ades'] = pd.factorize(df_flight['adep_ades'])[0]

    # Save the updated DataFrame back to the CSV file
    df_flight.to_csv(file, index=False)

    print(f"Updated file: {file} with new 'adep_ades' column.")

print("All files have been updated with the new 'adep_ades' column.")

/tmp/ipykernel_5393/312833122.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_flight['adep_ades'] = df_flight[['adep', 'ades']].apply(lambda x: f'{x[0]}_{x[1]}', axis=1)


Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv with new 'adep_ades' column.


/tmp/ipykernel_5393/312833122.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_flight['adep_ades'] = df_flight[['adep', 'ades']].apply(lambda x: f'{x[0]}_{x[1]}', axis=1)


Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv with new 'adep_ades' column.
All files have been updated with the new 'adep_ades' column.


# Great Circle distance Adep / Ades

In [12]:
#JGO: To refactor in order to calculate only one time each ADEP/ADES pair: DONE
import pandas as pd
from traffic.data import airports
from geopy.distance import great_circle

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Function to calculate great circle distance
def calculate_great_circle_distance(row, adep_latlon, ades_latlon):
    adep_coords = adep_latlon.get(row['adep'], None)
    ades_coords = ades_latlon.get(row['ades'], None)
    if adep_coords and ades_coords:
        return great_circle(adep_coords, ades_coords).kilometers
    else:
        return None

# Loop through each file
for file in files_to_process:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Get unique departure and arrival airport codes
    unique_adeps = df['adep'].unique()
    unique_ades = df['ades'].unique()

    # Create dictionaries to store lat/lon of airports
    adep_latlon = {}
    ades_latlon = {}

    # Retrieve lat/lon for unique departure airports
    for adep in unique_adeps:
        try:
            airport = airports[adep]
            adep_latlon[adep] = airport.latlon
        except ValueError:
            adep_latlon[adep] = None

    # Retrieve lat/lon for unique arrival airports
    for ades in unique_ades:
        try:
            airport = airports[ades]
            ades_latlon[ades] = airport.latlon
        except ValueError:
            ades_latlon[ades] = None
    tmp=df.groupby(["adep","ades"]).count().reset_index()
    tmp=tmp[["adep","ades"]]
    # Calculate great circle distances
    tmp['great_circle_distance_adep_ades']=tmp.apply(calculate_great_circle_distance, axis=1, 
                                                      args=(adep_latlon, ades_latlon))
    df=df.merge(tmp,on=["adep","ades"])
    #df['great_circle_distance_adep_ades'] = df.apply(calculate_great_circle_distance, axis=1, 
    #                                                  args=(adep_latlon, ades_latlon))

    # Save the modified DataFrame back to the CSV file
    df.to_csv(file, index=False)

    print(f"Updated file: {file} with great circle distances.")

print("Great circle distances have been calculated and saved for all files.")


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv with great circle distances.
Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv with great circle distances.
Great circle distances have been calculated and saved for all files.


# Flown_distance / Great_circle

In [13]:
import pandas as pd

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Loop through each file
for file in files_to_process:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Convert flown_distance from nautical miles to kilometers
    # 1 nautical mile = 1.852 kilometers
    df['flown_distance_km'] = df['flown_distance'] * 1.852

    # Calculate the great circle flown distance
    df['great_circle_flown_distance'] = df['flown_distance_km'] / df['great_circle_distance_adep_ades']

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file, index=False)

    print(f"Updated file: {file} with new 'flown_distance_km' and 'great_circle_flown_distance' columns.")

print("All files have been updated with the new calculations.")


Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv with new 'flown_distance_km' and 'great_circle_flown_distance' columns.
Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv with new 'flown_distance_km' and 'great_circle_flown_distance' columns.
All files have been updated with the new calculations.


# Average_speed ²

In [14]:
import pandas as pd

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Function to calculate new columns based on time data
def calculate_takeoff_time_duration(df):
    # 1. Calculate takeoff_time (actual_offblock_time + taxiout_time)
    df['takeoff_time'] = df['actual_offblock_time'] + pd.to_timedelta(df['taxiout_time'], unit='m')

    # 2. Calculate duration in minutes (arrival_time - takeoff_time)
    df['duration'] = (df['arrival_time'] - df['takeoff_time']).dt.total_seconds() / 60  # Convert to minutes

    # 3. Calculate average_speed (flown_distance / duration)
    df['average_speed'] = (df['flown_distance'] / df['duration']) * 60  # Convert to nautical miles per hour

    df['average_speed'] = df['average_speed'] ** 2

    return df

# Loop through each file
for file in files_to_process:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file)

    # Convert date columns to datetime format
    df['actual_offblock_time'] = pd.to_datetime(df['actual_offblock_time'])
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])

    # Apply the calculations to the DataFrame
    df = calculate_takeoff_time_duration(df)

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file, index=False)

    print(f"Updated file: {file} with new columns.")

print("All files have been updated with new columns.")


Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv with new columns.
Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv with new columns.
All files have been updated with new columns.


# Manage offblock time and arrival time

In [15]:
import pandas as pd

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Function to determine the season based on the month
def determine_season(month):
    if month in [12, 1, 2]:
        return 1  # Winter
    elif month in [3, 4, 5]:
        return 2  # Spring
    elif month in [6, 7, 8]:
        return 3  # Summer
    elif month in [9, 10, 11]:
        return 4  # Autumn

# Loop through each file
for file in files_to_process:
    # Read the file
    df = pd.read_csv(file)

    # Ensure the 'actual_offblock_time' and 'arrival_time' columns are in datetime format
    df['actual_offblock_time'] = pd.to_datetime(df['actual_offblock_time'])
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])

    # Extract month, day, and hour from actual_offblock_time
    df['month_offblock_time'] = df['actual_offblock_time'].dt.month
    df['day_offblock_time'] = df['actual_offblock_time'].dt.day
    df['hour_offblock_time'] = df['actual_offblock_time'].dt.hour

    # Extract month, day, and hour from arrival_time
    df['month_arrival_time'] = df['arrival_time'].dt.month
    df['day_arrival_time'] = df['arrival_time'].dt.day
    df['hour_arrival_time'] = df['arrival_time'].dt.hour

    # Apply the function to create the 'season' column
    df['season'] = df['month_offblock_time'].apply(determine_season)

    # Save the modified DataFrame back to the same file
    df.to_csv(file, index=False)

print("The season, month, day, and hour columns have been added and saved to both files.")


The season, month, day, and hour columns have been added and saved to both files.


In [16]:
import pandas as pd

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Loop through each file
for file in files_to_process:
    # Read the data
    df = pd.read_csv(file)

    # Ensure the 'actual_offblock_time' column is in datetime format
    df['actual_offblock_time'] = pd.to_datetime(df['actual_offblock_time'])

    # Extract the hour from the 'actual_offblock_time' column
    df['hour_offblock_time'] = df['actual_offblock_time'].dt.hour

    # Define a function to assign the time of day period
    def determine_day_period(hour):
        if hour < 9:
            return 1  # Before 9 AM
        elif 9 <= hour <= 16:
            return 2  # Between 9 AM and 4 PM
        else:
            return 3  # After 4 PM

    # Apply this function to create the 'day_period' column
    df['day_period'] = df['hour_offblock_time'].apply(determine_day_period)

    # Save the file with the new column
    df.to_csv(file, index=False)

    print(f"Updated file: {file}")

print("The 'hour_offblock_time' and 'day_period' columns have been added and saved to both files.")


Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv
Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv
The 'hour_offblock_time' and 'day_period' columns have been added and saved to both files.


# Aircraft range category

In [17]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Define a function to assign distance intervals
def get_flown_distance_interval(distance):
    if distance < 500:
        return "0-500"
    elif 500 <= distance < 1000:
        return "500-1000"
    elif 1000 <= distance < 1500:
        return "1000-1500"
    elif 1500 <= distance < 2000:
        return "1500-2000"
    elif 2000 <= distance < 4000:
        return "2000-4000"
    else:
        return "4000+"

# Loop through each file
for file in files_to_process:
    # Read the flight data
    df_flight = pd.read_csv(file)

    # Apply the function to create the flown_distance_interval column
    df_flight['range_category'] = df_flight['flown_distance'].apply(get_flown_distance_interval)

    # Use LabelEncoder to create the range_category_encoded column
    le = LabelEncoder()
    df_flight['range_category_encoded'] = le.fit_transform(df_flight['range_category'])

    # Save the modified DataFrame back to the same file
    df_flight.to_csv(file, index=False)

    print(f"Updated file: {file}")

print("The 'range_category' and 'range_category_encoded' columns have been added and saved to both files.")

Updated file: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv
Updated file: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv
The 'range_category' and 'range_category_encoded' columns have been added and saved to both files.


# AOBT seconds, days with sinus/cosinus
# Speed and wind angle sinus/cosinus

In [18]:
import pandas as pd
import numpy as np

# List of files to process
files_to_process = [os.path.join(source_data_folder,challenge_file_preproc), os.path.join(source_data_folder,submission_file_preproc)]

# Loop through each file
for file in files_to_process:
    # Load the flight data from the CSV file into a DataFrame
    df_flight = pd.read_csv(file)

    # Convert the 'actual_offblock_time' column to datetime format
    df_flight['actual_offblock_time_dt'] = pd.to_datetime(df_flight['actual_offblock_time'])

    # Calculate the number of seconds since the start of the day for 'actual_offblock_time'
    df_flight['aobt_seconds'] = (df_flight['actual_offblock_time_dt'] - df_flight['actual_offblock_time_dt'].dt.normalize()) / pd.Timedelta('1 second')

    # Compute the cosine of the seconds since the start of the day for cyclic feature representation
    df_flight['aobt_seconds_cos'] = np.cos(df_flight['aobt_seconds'] * (np.pi * 2 / (3600 * 24)))

    # Compute the sine of the seconds since the start of the day for cyclic feature representation
    df_flight['aobt_seconds_sin'] = np.sin(df_flight['aobt_seconds'] * (np.pi * 2 / (3600 * 24)))

    # Calculate the number of days since a reference date (January 1, 2022) for 'actual_offblock_time'
    df_flight['aobt_days'] = (df_flight['actual_offblock_time_dt'] - pd.to_datetime("2022-01-01 00:00+00")) / pd.Timedelta('1 day')

    # Compute the cosine of the days since the reference date for cyclic feature representation
    df_flight['aobt_days_cos'] = np.cos(df_flight['aobt_days'] * (np.pi * 2 / (365)))

    # Compute the sine of the days since the reference date for cyclic feature representation
    df_flight['aobt_days_sin'] = np.sin(df_flight['aobt_days'] * (np.pi * 2 / (365)))

    df_flight["wind_track_angle_cos"]=np.cos((np.pi/180)*(df_flight["ground_wind_direction"]-df_flight["ground_airspeed_angle"]))
    df_flight["wind_track_angle_sin"]=np.sin((np.pi/180)*(df_flight["ground_wind_direction"]-df_flight["ground_airspeed_angle"]))

    df_flight["plateau_wind_track_angle_cos"]=np.cos((np.pi/180)*(df_flight["plateau_wind_direction"]-df_flight["plateau_airspeed_angle"]))
    df_flight["plateau_wind_track_angle_sin"]=np.sin((np.pi/180)*(df_flight["plateau_wind_direction"]-df_flight["plateau_airspeed_angle"]))

    # Save the modified DataFrame back to the same CSV file
    df_flight.to_csv(file, index=False)

    print(f"Processed and saved: {file}")


Processed and saved: /workspaces/PRCDataChallenge/data/challenge_set_preproc.csv
Processed and saved: /workspaces/PRCDataChallenge/data/submission_set_preproc.csv


# Add ICAO code, and ICAO couple
IGNORE - REQUIRES DATA NOT MADE AVAILABLE (COULD NOT BE OPEN)

In [21]:
import pandas as pd

# Loading the files
utc_file =  os.path.join(source_data_folder,os.getenv("UTC_OFFSET"))

file_1 = os.path.join(source_data_folder,challenge_file_preproc)
file_2 = os.path.join(source_data_folder,submission_file_preproc)

utc_df = pd.read_csv(utc_file)
utc_df["ICAO_prefix"]=utc_df['country_code_adep'].astype(str).str[0]
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)

# Merge with the 'UtcToLocalTimeOffset.csv' file to obtain 'ICAO_prefix' based on 'country_code_adep'
df1_merged = pd.merge(df1, utc_df[['country_code_adep', 'ICAO_prefix']], left_on='country_code_adep', right_on='country_code_adep', how='left')
df2_merged = pd.merge(df2, utc_df[['country_code_adep', 'ICAO_prefix']], left_on='country_code_adep', right_on='country_code_adep', how='left')

# Rename the column 'ICAO_prefix' to 'country_code_adep_icao'
df1_merged.rename(columns={'ICAO_prefix': 'country_code_adep_icao'}, inplace=True)
df2_merged.rename(columns={'ICAO_prefix': 'country_code_adep_icao'}, inplace=True)

# Add the column "country_code_ades_icao" based on "ades" and their corresponding ICAO code
df1_merged = pd.merge(df1_merged, utc_df[['country_code_adep', 'ICAO_prefix']], left_on='country_code_ades', right_on='country_code_adep', how='left')
df2_merged = pd.merge(df2_merged, utc_df[['country_code_adep', 'ICAO_prefix']], left_on='country_code_ades', right_on='country_code_adep', how='left')

# Rename the second column 'ICAO_prefix' to 'country_code_ades_icao'
df1_merged.rename(columns={'ICAO_prefix': 'country_code_ades_icao'}, inplace=True)
df2_merged.rename(columns={'ICAO_prefix': 'country_code_ades_icao'}, inplace=True)

# Remove duplicate columns after the merge
df1_merged.drop(columns=['country_code_adep_y'], inplace=True)
df2_merged.drop(columns=['country_code_adep_y'], inplace=True)

# Save the updated files
df1_merged.to_csv(file_1, index=False)
df2_merged.to_csv(file_2, index=False)

print("The updated files have been saved with the columns 'country_code_adep_icao' and 'country_code_ades_icao'.")


The updated files have been saved with the columns 'country_code_adep_icao' and 'country_code_ades_icao'.


In [22]:
import pandas as pd

# Loading the cleaned files
file_1 = os.path.join(source_data_folder,challenge_file_preproc)
file_2 = os.path.join(source_data_folder,submission_file_preproc)

df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)

# Function to generate the 'country_code_icao_couple' column
def create_icao_couple(row):
    adep_first_letter = str(row['country_code_adep_icao'])[0] if pd.notna(row['country_code_adep_icao']) else ''
    ades_first_letter = str(row['country_code_ades_icao'])[0] if pd.notna(row['country_code_ades_icao']) else ''
    
    # Sort the letters in alphabetical order
    sorted_letters = ''.join(sorted([adep_first_letter, ades_first_letter]))
    return sorted_letters

# Apply the function on both files
df1['country_code_icao_couple'] = df1.apply(create_icao_couple, axis=1)
df2['country_code_icao_couple'] = df2.apply(create_icao_couple, axis=1)

# Save the files with the new column
df1_merged.to_csv(file_1, index=False)
df2_merged.to_csv(file_2, index=False)

print("The files have been updated with the 'country_code_icao_couple' column.")


The files have been updated with the 'country_code_icao_couple' column.


# Taxi_in

In [34]:
import pandas as pd

# Load the two CSV files into DataFrames
airport_db = pd.read_csv('./data/MyAirportDatabase_Updated.csv')
runways_db = pd.read_csv('./data/runways.csv')

# Merge the two DataFrames on the 'airport' and 'airport_ident' columns
merged_df = pd.merge(airport_db, runways_db[['airport_ident', 'length_ft']], 
                     left_on='airport', right_on='airport_ident', 
                     how='left', suffixes=('', '_runway'))

# Fill the 'MinRWYlength' column with values from 'length_ft'
merged_df['MinRWYlength'] = merged_df['length_ft']

# Drop the intermediate 'length_ft' and 'airport_ident' columns if necessary
merged_df.drop(columns=['length_ft', 'airport_ident'], inplace=True)

# Save the modified DataFrame to a new CSV file
merged_df.to_csv('./database/MyAirportDatabase_Updated.csv', index=False)

print("The 'MinRWYlength' column has been updated with runway lengths.")


ParserError: Error tokenizing data. C error: Expected 1 fields in line 14225, saw 110


In [36]:
import pandas as pd

# Load the updated file into a DataFrame
updated_airport_db = pd.read_csv('./data/MyAirportDatabase_Updated.csv')

# Check if the 'Taxi_In' column exists
if 'Taxi_In' in updated_airport_db.columns:
    # Create the new column 'taxi_in_seconds'
    updated_airport_db['taxi_in_seconds'] = updated_airport_db['Taxi_In'] * 60

    # Save the modified DataFrame back to the same CSV file
    updated_airport_db.to_csv('./data/MyAirportDatabase_Updated.csv', index=False)

    print("The 'taxi_in_seconds' column has been added successfully.")
else:
    print("The 'Taxi_In' column does not exist in the DataFrame.")


OSError: Cannot save file into a non-existent directory: 'database'

In [40]:
import pandas as pd

# Load the files into DataFrames
challenge_set = os.path.join(source_data_folder,challenge_file_preproc)
final_submission = os.path.join(source_data_folder,submission_file_preproc)

# Load the file with the 'taxi_in_seconds' column
airport_db = pd.read_csv('./database/MyAirportDatabase_Updated.csv')

# Check if the 'taxi_in_seconds' column exists
if 'taxi_in_seconds' in airport_db.columns:
    # Remove duplicates while keeping the first occurrence of each airport
    airport_db_unique = airport_db[['airport', 'taxi_in_seconds']].drop_duplicates(subset='airport')

    # Add taxi_in_seconds to the final_submission file
    final_submission_merged = pd.merge(final_submission, airport_db_unique, 
                                        left_on='ades', right_on='airport', 
                                        how='left', suffixes=('', '_airport'))

    # Add taxi_in_seconds to the challenge_set_test_2 file
    challenge_set_merged = pd.merge(challenge_set, airport_db_unique, 
                                     left_on='ades', right_on='airport', 
                                     how='left', suffixes=('', '_airport'))

    # Save the modified files
    final_submission_merged.to_csv(os.path.join(source_data_folder,submission_file_preproc), index=False)
    challenge_set_merged.to_csv(os.path.join(source_data_folder,challenge_file_preproc), index=False)

    print("The 'taxi_in_seconds' columns have been added to the files successfully.")
else:
    print("The 'taxi_in_seconds' column does not exist in the airport DataFrame.")


FileNotFoundError: [Errno 2] No such file or directory: './database/MyAirportDatabase_Updated.csv'

In [38]:
import pandas as pd

# Load the files into DataFrames
# Loading the cleaned files
final_challenge = os.path.join(source_data_folder,challenge_file_preproc)
final_submission = os.path.join(source_data_folder,submission_file_preproc)

# Define a function to calculate taxi_in
def calculate_taxi_in(row):
    if row['aircraft_category'] == 'Heavy':
        return row['taxi_in_seconds'] * 0.3
    elif row['aircraft_category'] == 'Medium':
        return row['taxi_in_seconds'] * 0.2
    elif row['aircraft_category'] == 'Light':
        return row['taxi_in_seconds'] * 0.1
    else:
        return None  # Default value if no condition is met

# Apply the function to create the taxi_in column
final_submission['taxi_in'] = final_submission.apply(calculate_taxi_in, axis=1)
challenge_set['taxi_in'] = challenge_set.apply(calculate_taxi_in, axis=1)

# Save the modified files
final_submission.to_csv(os.path.join(source_data_folder,challenge_file_preproc), index=False)
challenge_set.to_csv(os.path.join(source_data_folder,submission_file_preproc), index=False)

print("The 'taxi_in' column has been added successfully to the files.")


AttributeError: 'str' object has no attribute 'apply'

# Runway_length

In [ ]:
import pandas as pd

# Load the files into DataFrames
final_submission = pd.read_csv('./final_submission_set.csv')  # Use the updated file
challenge_set = pd.read_csv('./challenge_set_test_2.csv')

# Load the file with MinRWYlength to get the runway lengths
airport_db = pd.read_csv('./database/MyAirportDatabase_Updated.csv')

# Extract the first non-null value of MinRWYlength for each airport
min_rwy_length_mapping = airport_db.groupby('airport')['MinRWYlength'].first().dropna()

# Convert runway length from feet to kilometers
min_rwy_length_mapping_km = min_rwy_length_mapping * 0.0003048  # 1 foot = 0.0003048 km

# Create a new column runway_length in the DataFrames using the mapping
final_submission['runway_length'] = final_submission['ades'].map(min_rwy_length_mapping_km)
challenge_set['runway_length'] = challenge_set['ades'].map(min_rwy_length_mapping_km)

# Save the modified files
final_submission.to_csv('./final_submission_set_with_runway_length.csv', index=False)
challenge_set.to_csv('./challenge_set_test_2_with_runway_length.csv', index=False)

print("The 'runway_length' column has been added successfully to the files.")


# Low_cost Airport

In [ ]:
import pandas as pd

# Charger les fichiers
final_submission = pd.read_csv("./final_submission_set.csv")
challenge_set_test = pd.read_csv("./challenge_set_test_2.csv")
low_cost_airports = pd.read_csv("./database/low-cost_Airports_Europe.csv")

# Obtenir la liste des codes ICAO des aéroports à bas coûts
low_cost_icao_codes = set(low_cost_airports['ICAO Code'].dropna())

# Fonction pour attribuer 1 ou 0 à low_cost_airport
def assign_low_cost_airport(row):
    if row['adep'] in low_cost_icao_codes or row['ades'] in low_cost_icao_codes:
        return 1
    else:
        return 0

# Appliquer la fonction à chaque fichier
final_submission['low_cost_airport'] = final_submission.apply(assign_low_cost_airport, axis=1)
challenge_set_test['low_cost_airport'] = challenge_set_test.apply(assign_low_cost_airport, axis=1)

# Enregistrer les fichiers modifiés
final_submission.to_csv("./final_submission_set_1.csv", index=False)
challenge_set_test.to_csv("./challenge_set_test_2_1.csv", index=False)

print("Colonnes 'low_cost_airport' ajoutées et fichiers sauvegardés.")
